# Training
Seems like we're ready for training!

In [16]:
import sys, os, datetime
sys.path.append('/home/andrea/AI/ispr_yolo/data')
sys.path.append('/home/andrea/AI/ispr_yolo/model')

from yolo import YOLOv3
from lossfunction import YoloLoss
from DataPreprocessing import create_dataset
import tensorflow as tf

#### Creating the Dataset

In [17]:
img_dir = '/home/andrea/AI/ispr_yolo/data/dataset_bdd/images/100k/train'+ '/*.jpg'
train_ds = create_dataset(img_dir)

#### Compiling the model

In [18]:
yolo = YOLOv3(size = 1280, training = True)

In [19]:
yolo_loss = YoloLoss(10)
yolo.compile(loss=[yolo_loss]*3, optimizer='nadam')
%load_ext tensorboard
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)
yolo.fit(train_ds,
         epochs = 100,
         callbacks = [tensorboard_callback])

%tensorboard --logdir logs

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard
Epoch 1/100


ResourceExhaustedError:  OOM when allocating tensor with shape[32,32,1280,1280] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node YOLOv3/darknet/conv2d_225/Conv2D (defined at <ipython-input-19-e94f4cf39d7c>:6) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.
 [Op:__inference_train_function_62729]

Function call stack:
train_function
